In [2]:
import time
import requests
from bs4 import BeautifulSoup
import pymongo

In [3]:
mc = pymongo.MongoClient()  # Connect to the MongoDB server using default settings
db = mc['strain_reviews']  # Use (or create) a database called 'election_predictions'
docs = db['review']  # Use (or create) a collection called 'docs'

In [6]:
docs.count_documents(filter={})

0

In [5]:
docs.drop()

## Here is how i want the scrape to work

    """LOS = [List of Strains]
        for s in LOS:
        url = "https://www.leafly.com/{}/reviews?page=".format(s)
        for i in range(1,100):
            d = get_reviews(url+str(i))
            st, rev = parse_docs(d)
            if len(st) > 0:
               append.(add to your dictionary)
            else:
                break"""

In [7]:
def table_check2(l):
    """This checks to see if a url is already in my database of reviews. 
    Only needs to be run in the get reviews function"""
    for url in l:
        if docs.find_one({'url':url}):
            None
        else:
            docs.insert_one({'url': url,
                 'html': requests.get(url).content
                 })

In [14]:
def get_reviews2(l):
    """Pass in a list of URL's and return them in a mongo db table as a dicitonary with 
    {'url', 'html'} and their corresponding values"""
    r = requests.get(l)
    html = (r.content)
    return html

In [9]:
def get_stars_list(d):
    stars = []
    for key, values in d.items():
        soup = BeautifulSoup(values, 'html.parser')
        tags = soup.select("div.div.stars")
        for t in tags:
            stars.append(t.attrs['style'])
    return stars

In [10]:
def star_int_conv(l):
    flat_stars= []
    star_num = []
    for s in l:
        star = int((s[6:].split(';')[0]).strip('px'))
        star_num.append((star/22))
    return star_num

In [11]:
def list_o_strains(i):
    LOS = []
    r = requests.get(i)
    soup2 = BeautifulSoup(r.content, 'html.parser')
    strains = soup2.find_all('a', class_="ga_Explore_Strain_Tile")
    for s in strains:
        LOS.append(s.attrs['href'])
    return LOS

In [12]:
def parse_docs2(d):
    """Parse the HTML docs that we have stored in a dictionary, return as a list.
    Also scrape and parse star rating for each review """
    strain_text= []
    star_rate = []
    user_name = []
    soup = BeautifulSoup(d, 'html.parser')
    revs = soup.find_all('p',class_='strain-review__text') 
    for r in revs:
        strain_text.append(r.text)
    tags = soup.select("div.div.stars")
    star = []
    for t in tags:
        star.append(t.attrs['style'])
    star_rate.append(star_int_conv(star))
    users = soup.find_all('div', class_='strain-review__title')
    for u in users:
        temp = u.find('h2')
        user_name.append(temp.text)
    return star_rate, strain_text, user_name

In [13]:
#Test works better, is updated!
def scraper_dummy1(los):
    los_urls = []
    rev_urls = []
    for s in los:
        url = "https://www.leafly.com{}/reviews?page=".format(s)
        print(url)
        for i in range(1,100):
            rev_urls=url+str(i)
            d = get_reviews2(rev_urls)
            star, reviews, users = parse_docs2(d)
            print(users)
            if len(star[0]) == 0:
                break

In [1]:
def scraper_dummy_test(los):
    los_urls = []
    rev_urls = []
    user_list_of_lists = []
    star_list_of_lists = []
    reviews_list_o_lists = []
    for s in los:
        url = "https://www.leafly.com{}/reviews?page=".format(s)
        strain_star_list = []
        strain_user_list = []
        strain_rev_list = []
        for i in range(1,100):
            rev_urls=url+str(i)
            d = get_reviews2(rev_urls)
            star, reviews, users = parse_docs2(d)
            star_unlisted = [x for x in star[0]]
            strain_star_list.extend(star_unlisted)
            strain_user_list.extend(users)
            strain_rev_list.extend(reviews)
            if len(star[0]) == 0:
                break
        star_list_of_lists.append(strain_star_list)
        user_list_of_lists.append(strain_user_list)
        reviews_list_o_lists.append(strain_rev_list)
    return  user_list_of_lists, reviews_list_o_lists, star_list_of_lists

In [161]:
test_strains = list_o_strains('https://www.leafly.com/explore')

In [162]:
test_strains[6]

'/hybrid/original-glue'

In [163]:
test = scraper_dummy_test(test_strains[6:8])
test

https://www.leafly.com/hybrid/original-glue/reviews?page=
https://www.leafly.com/hybrid/white-widow/reviews?page=


([['chronicallymedicated',
   'MZ_PHD_in_420',
   'gatorhunts420',
   'calebargh',
   'purpleurklealienchempantsdogkush',
   'chroniccannabisconsumsr',
   'mrobadamus',
   'Budsey420',
   'Munchiez2',
   'chroniccannabisconsumsr',
   'ChronicComic',
   'Greenleaf_Compton',
   'Hookakat1',
   'ourlife',
   'LegitLogic',
   'JustinCider',
   'flowerbud420',
   'jpersch',
   'KindGoddess420',
   'OnePDXOne',
   'sirflingspoo',
   '420life',
   'Free2beme',
   'Donnie1967',
   'Ogflight2space',
   'warriorforbeauty',
   'chrisg3121',
   'thejuju',
   'Sammi_S',
   'redasalobster',
   'tmaass4',
   'DankNunuFresh',
   'EssTee5ive',
   'PipeDreamer4',
   'quentello',
   'vanugrah',
   'PumpkinGemini11',
   'tinfoil',
   'funkmasterastronaut',
   'BaronOfBud',
   'TheWidowsSon226',
   'GooglyDancingBear',
   'Stu35846',
   'SalmonCreekBigBuds',
   'plschoosenewusername',
   'LegitLogic',
   'nevildog',
   'MGomsrud',
   'cogollohazer',
   'RWright',
   'bge1342',
   'bosieboy',
   'dbvapor',


In [ ]:
a = [[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]]
b = [[5.0, 5.0, 4.0, 5.0, 4.0, 4.0, 5.0, 5.0]]

In [72]:
[x for x in b[0]]

[5.0, 5.0, 4.0, 5.0, 4.0, 4.0, 5.0, 5.0]

In [80]:
i = len(a)
[x for x in a[i]]

IndexError: list index out of range

In [81]:
pd. 

2